In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

## Question 1

In [5]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [6]:
len(df)

1154112

## Question 2

In [7]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
df['duration'].mean()

19.167224093791006

## Question 3

In [9]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [10]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)

In [11]:
(df['PUlocationID'] == -1).sum() / len(df) * 100

83.52732770722618

## Question 4

In [12]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

In [13]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [20]:
print(X_train.shape[1])

525


## Question 5

In [15]:
target = 'duration'
y_train = df[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.528519107211832

## Question 6

In [17]:
df_val = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

df_val['duration'] = df_val.dropOff_datetime - df_val.pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

df_val['PUlocationID'] = df_val['PUlocationID'].fillna(-1)
df_val['DOlocationID'] = df_val['DOlocationID'].fillna(-1)

categorical = ['PUlocationID', 'DOlocationID']

df_val[categorical] = df_val[categorical].astype(str)

val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

In [18]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(1109826, 525) (1109826,)
(990113, 525) (990113,)


In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.014283203471416